In [1]:
import requests
from bs4 import BeautifulSoup
import json
import re
import os
from requests_html import HTMLSession
import time
import itertools
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
session = HTMLSession()

In [2]:
def write_to_json(lst, fn):
    import json
    with open(fn, "a") as file:
        for item in lst:
            x = json.dumps(item, ensure_ascii=False)
            file.write(x + "\n")

In [4]:
category = 'english'

with open(f"output/lyric_{category}.txt") as f:
    lst = f.readlines()

In [5]:
max_workers = 70

def process_url(x):
    url = x.strip()
    main_url = "https://www.lyrics.my"
    #print(f"start crawl: {url}")
    
    r = session.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    
    with open("debug.html", "w") as debug:
        debug.write(str(soup))
    
    lyrics_list = soup.find_all("div", {"class": "lyrics-list"})
    
    for item in lyrics_list:
        songs = item.find_all("div", {"class": "data video"})
        for item in songs:
            name = item.find("span", {"class": "name"}).text
            title = item.find("span", {"class": "title"}).text
            url_song = f"{main_url}{item.find('a').get('href')}"

            data = {
            "name": name,
            "title": title,
            "url": url_song,
            "search_page": url
            }

            write_to_json([data], f"output/lyrics-main-artist-{category}_test.json")

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_url, x) for x in lst]
    
    for future in tqdm(futures, total=len(lst)):
        result = future.result()
        

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5238/5238 [00:54<00:00, 96.31it/s]


# get artist link (char)

In [ ]:
for i in tqdm(list(map(chr, range(97, 123)))):
    page = i
    category = 'english'
    
    url = f"https://www.lyrics.my/categories/{category}/char/{page}"
    main_url = "https://www.lyrics.my"

    print(f"start crawl: {url}")
    
    r = session.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    
    artist = soup.find_all("div", {"class": "artist-list"})
    
    artist = soup.find_all("div", {"class": "artist-list"})

    all_artist = [item.find_all("a") for item in artist]
    all_artist = list(itertools.chain(*all_artist))

    lst_lnk = [f"{main_url}{i.get('href')}" for i in all_artist]

    file = open(f'output/lyric_{category}_test.txt','a')
    for item in lst_lnk:
        file.write(item + "\n")

    file.close()

# get link from artist page

In [ ]:
category = 'nasyid'

with open(f"output/lyric_{category}.txt") as f:
    lst = f.readlines()

In [ ]:
for i in lst:
    url = i.strip()
    main_url = "https://www.lyrics.my"
    print(f"start crawl: {url}")
    
    r = session.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    
    with open("debug.html", "w") as debug:
        debug.write(str(soup))
    
    lyrics_list = soup.find_all("div", {"class": "lyrics-list"})
    
    lyrics_url = []
    
    for item in lyrics_list:
        songs = item.find_all("div", {"class": "data video"})
        for item in songs:
            name = item.find("span", {"class": "name"}).text
            title = item.find("span", {"class": "title"}).text
            url_song = f"{main_url}{item.find('a').get('href')}"

            data = {
            "name": name,
            "title": title,
            "url": url_song,
            "search_page": url
            }

            write_to_json([data], f"output/lyrics-main-artist-{category}.json")
#     time.sleep(1)

# get lyrics

In [ ]:
category = 'nasyid'

with open(f"output/lyrics-main-artist-{category}.json") as f:
    lst = f.readlines()
    
for item in lst:
    item = json.loads(item)
    
    url = item['url']
    name = item['name']
    title = item['title']
    
    print(f"start crawl: {url}")
    
    r = session.get(url)
    soup = BeautifulSoup(r.content, "lxml")

    lyrics_list = soup.find("div", {"class": "show_lyric"})

    lyrics = []
    
    for p in lyrics_list.find_all("span"):
        if "taken from" in p.text:
            continue
        lyrics.append((p.text).strip())

    lyric = " ".join(lyrics)
    
    data = {
        "text": lyric,
        "url": url,
        "name": name,
        "title": title
    }
    
    write_to_json([data], f"output/lyrics_{category}.json")

# get link (main)

In [ ]:
start_page = 1
end_page = 10

while True:
    if start_page > end_page:
        break
        
    url = f"https://www.lyrics.my/page/{start_page}"
    main_url = "https://www.lyrics.my"
    
    print(f"start crawl: {url}")
    
    r = session.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    
    with open("debug.html", "w") as debug:
        debug.write(str(soup))
    
    lyrics_list = soup.find_all("div", {"class": "lyrics-list"})
    
    lyrics_url = []
    
    for item in lyrics_list:
        songs = item.find_all("div", {"class": "data video"})
        for item in songs:
            name = item.find("span", {"class": "name"}).text
            title = item.find("span", {"class": "title"}).text
            url_song = f"{main_url}{item.find('a').get('href')}"

            data = {
            "name": name,
            "title": title,
            "url": url_song,
            "search_page": url
            }

            write_to_json([data], f"output/lyrics-main.json")
        
    
    start_page += 1
    
    time.sleep(1)
